In [1]:
import random
from collections import defaultdict
import hashlib

# Load paragraphs from a file
def loadParagraphs(file_path):
    with open(file_path, 'r') as file:
        paragraphs = file.readlines()
    return [p.strip() for p in paragraphs if p.strip()]

# Generate k-shingles from a paragraph
def kShingles(paragraph, k):
    words = paragraph.split()
    shingles = set()
    for i in range(len(words) - k + 1):
        shingle = ' '.join(words[i:i + k])
        shingles.add(shingle)
    return shingles

# Hash shingles using SHA-256 and reduce them to 4-digit integers
def hashShingles(shingles):
    return {
        int(hashlib.sha256(shingle.encode()).hexdigest(), 16) % 10000 for shingle in shingles
    }

# Generate MinHash signatures
def minhashSignatures(sortedPairs, n):
    signatures = {
        doc_id: [float('inf')] * n for doc_id in sortedPairs.keys()
    }
    for i in range(n):
        randomHash = random.sample(range(1, 10001), len(sortedPairs))
        for doc_id, shingle_list in sortedPairs.items():
            for shingle in shingle_list:
                if signatures[doc_id][i] > randomHash[shingle % len(randomHash)]:
                    signatures[doc_id][i] = randomHash[shingle % len(randomHash)]
    return signatures

# Generate candidate pairs using Locality-Sensitive Hashing (LSH)
def candidatePairs(signatures, b, r):
    buckets = defaultdict(list)
    for doc_id, signature in signatures.items():
        for i in range(b):
            band = tuple(signature[i * r:(i + 1) * r])
            buckets[band].append(doc_id)
    return buckets

# Calculate Jaccard similarity between two sets of shingles
def jaccardSimilarity(shinglesA, shinglesB):
    intersection = len(shinglesA.intersection(shinglesB))
    union = len(shinglesA.union(shinglesB))
    return intersection / union if union != 0 else 0

# File path to the dataset
file_path = "Project2_dataset_similarity.txt"

# Load and process paragraphs
paragraphs = loadParagraphs(file_path)

# Shingling and hashing parameters
k = 5
shingleDict = {i: kShingles(paragraph, k) for i, paragraph in enumerate(paragraphs)}
hashedShingles = {i: hashShingles(shingles) for i, shingles in shingleDict.items()}

# MinHash parameters
n = 100
b = 3
r = 10

# Generate MinHash signatures and candidate pairs
signatures = minhashSignatures(hashedShingles, n)
candidates = candidatePairs(signatures, b, r)

# Identify similar pairs based on Jaccard similarity
similarPairs = []
threshold = 0.92
for bucket in candidates.values():
    if len(bucket) > 1:
        for i in range(len(bucket)):
            for j in range(i + 1, len(bucket)):
                shinglesA = shingleDict[bucket[i]]
                shinglesB = shingleDict[bucket[j]]

                similar = jaccardSimilarity(shinglesA, shinglesB)
                if similar >= threshold:
                    similarPairs.append((bucket[i], bucket[j], similar))

# Output similar pairs
for doc1, doc2, similar in similarPairs:
    print(f"Paragraphs {doc1} and {doc2} are similar with similarity {similar:.2f}")


Paragraphs 160 and 174 are similar with similarity 1.00
Paragraphs 160 and 188 are similar with similarity 1.00
Paragraphs 160 and 202 are similar with similarity 1.00
Paragraphs 160 and 216 are similar with similarity 1.00
Paragraphs 174 and 188 are similar with similarity 1.00
Paragraphs 174 and 202 are similar with similarity 1.00
Paragraphs 174 and 216 are similar with similarity 1.00
Paragraphs 188 and 202 are similar with similarity 1.00
Paragraphs 188 and 216 are similar with similarity 1.00
Paragraphs 202 and 216 are similar with similarity 1.00
Paragraphs 113 and 242 are similar with similarity 1.00
Paragraphs 113 and 242 are similar with similarity 1.00
Paragraphs 113 and 242 are similar with similarity 1.00
Paragraphs 117 and 245 are similar with similarity 1.00
Paragraphs 117 and 245 are similar with similarity 1.00
Paragraphs 117 and 245 are similar with similarity 1.00
Paragraphs 118 and 246 are similar with similarity 1.00
Paragraphs 118 and 246 are similar with similari

1.
Chose shingle size k = 5 to avoid common 2 or 3 word phrases. It represents a good trade-off between generating a lot of shingles if a smaller k was used while also generating enough to detect patterns among similar paragraphs.

2.
Naive comparison of Minhash signatures is extremely inefficient requiring a quadratic number of comparisons. The Banding Technique and LSH improves efficiency by focusing only on likely similar document pairs, filtering out pairs that are not likely to be similar.

3.
My analysis identified a large number of paragraph pairs that have identical similarity scores. For certain pairs they are indeed identical (119, 247) but I am not sure they are completely identical for every identified pair. Minhash signatures of certain paragraphs could be the same, but the actual paragraphs may not actually be identical.